In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as smp
import plotly.graph_objects as go
from scipy import constants
import random


# Define the parameters of the basic simulation


class KleinNishina:
    """Class for simulating Compton scattering using the Klein-Nishina equation."""
    
    def __init__(self, E, no_of_points=10000):
        """Initialize the Klein-Nishina simulator with the incident photon energy E."""
        self.E = E
        self._calculate_distribution(no_of_points)
    
    def _calculate_distribution(self, no_of_points):
        """Calculate the cumulative distribution function for scattering angles."""
        alp = (self.E * constants.e) / (constants.electron_mass * constants.c**2)
        r_e = 2.8179403262e-15  # Classical electron radius
        
        theta, alpha = smp.symbols('theta alpha', real=True, positive=True)
        dsdo = r_e / 2 * (1 + smp.cos(theta)**2) / (1 + alpha * (1 - smp.cos(theta)))**2 * \
               (1 + alpha**2 * (1 - smp.cos(theta))**2 / ((1 + smp.cos(theta)**2) * (1 + alpha * (1 - smp.cos(theta)))))
        dsdt = 2 * smp.pi * dsdo * smp.sin(theta)
        dsdt = dsdt.simplify()
        s = smp.integrate(dsdt, (theta, 0, smp.pi)).simplify().simplify()
        pdf_omega = dsdo / s
        pdf_omega = pdf_omega.simplify()
        pdf_omega_f = smp.lambdify([theta, alpha], pdf_omega)
        
        self.the = np.linspace(0, np.pi, no_of_points)
        self.pdf_o = pdf_omega_f(self.the, alp)
        self.cdf_o = np.cumsum(self.pdf_o) / np.sum(self.pdf_o)  # Normalize CDF
    
    def calc_scattering_angles(self):
        """Calculate scattering angles based on the Klein-Nishina distribution."""
        rand_o = np.random.rand()
        i = np.argmin(np.abs(self.cdf_o - rand_o))
        polar = self.the[int(i)]
        azimuthal = np.random.rand() * 2 * np.pi
        return  polar, azimuthal
   # def angle_x(self, polar):
        # x = np.rad2deg(self.the[i])
        # y = np.rad2deg(polar)
       # z = 180 -(x+y)
       # return z
    def angle_with_x_axis(self, polar, azimuthal):
        """
        Calculating the angle made by the scattered gamma photon with the x-axis.
        Parameters: polar : θ in radians, Azimuthal: φ in radians.
        Returns: The angle with the x-axis in radians.
        """
        x = np.sin(polar) * np.cos(azimuthal)
        y = np.sin(polar) * np.sin(azimuthal)
        z = np.cos(polar)
        magnitude = np.sqrt(x**2 + y**2 + z**2)
        cos_alpha = x / magnitude
        alpha = np.arccos(cos_alpha)
        return alpha
        


In [3]:
def electron_energy(E, angle_deg):
    """
    Calculate the kinetic energy of the recoiling electron after Compton scattering.
    """
    mec2 = 0.511  # Rest mass energy of the electron in MeV
    angle_rad = np.radians(angle_deg)
    E_prime = E / (1 + (E / mec2) * (1 - np.cos(angle_rad))) #E_prime = energy after scattering
    Ke = E - E_prime #  Ke = energy of the recoiling electron
    return Ke

In [11]:
def generate_source_points(num_sources, dimension, shape_type=None, x_min=-4, x_max=44, y_min=-4, y_max=44, radius=10, distance=10):
    source_positions = []

    if dimension == '3D':
        for _ in range(num_sources):
            angle = random.uniform(0, 2 * np.pi)
            elevation = random.uniform(-np.pi / 2, np.pi / 2)
            x = round(random.uniform(x_min, x_max), 2)
            y = round(random.uniform(y_min, y_max), 2)
            z = round(distance * np.sin(elevation), 2)
            source_positions.append([x, y, z])
    elif dimension == '2D':
        # Initialize empty arrays for points
        x_points = np.array([])
        y_points = np.array([])
         
        if shape_type == 'ring':
            # Generate points on the perimeter of a circle
            angles = np.linspace(0, 2 * np.pi, num_sources)
            x_points = radius * np.cos(angles)
            y_points = radius * np.sin(angles)
            
        elif shape_type == 'circle':
            # Generate points to cover the entire area of a circle
            angles = np.random.uniform(0, 2 * np.pi, num_sources)
            radii = np.sqrt(np.random.uniform(0, radius**2, num_sources))
            x_points = radii * np.cos(angles)
            y_points = radii * np.sin(angles)
            
        elif shape_type == 'square':
            # Generate points to cover the entire area of a square
            x_points = np.random.uniform(x_min, x_max, num_sources)
            y_points = np.random.uniform(y_min, y_max, num_sources)
            
        elif shape_type == 'random':
            # Generate random points in 2D plane at z = 0
            x_points = np.random.uniform(x_min, x_max, num_sources)
            y_points = np.random.uniform(y_min, y_max, num_sources)
            
        else:
            raise ValueError("Invalid shape_type. Choose 'ring', 'circle', 'square', or 'random'.")
        
        # Randomly translate the points to a position within the plane limits
        if shape_type in ['ring', 'circle']:
            x_center = np.random.uniform(x_min + radius, x_max - radius)
            y_center = np.random.uniform(y_min + radius, y_max - radius)
            x_points += x_center
            y_points += y_center
        
        # Add points to source positions with z = 0
        source_positions = list(zip(np.round(x_points, 2), np.round(y_points, 2), np.zeros(num_sources)))

    else:
        raise ValueError("Invalid dimension. Choose '2D' or '3D'.")

    return source_positions

In [17]:
# take a point in continuous space and map it to 
# the center of the nearest detector element within a detector grid. Each detector element is a square of a specified size.

def map_to_detector(point, detector_size):
    return (point // detector_size) * detector_size + detector_size / 2


In [ ]:
# Create an instance of the Klein-Nishina class - get a probability distribution
# to get angles from.
source_energy = 1000000 # eV
kn = KleinNishina(source_energy)
#source_dimension = '2D'
# Source position based on dimensionality
#if source_dimension == '2D':
#    distance = 20  # Example distance for 2D setup
#    angle = random.uniform(0, 2 * np.pi)
#    source_pos = [distance * np.cos(angle), distance * np.sin(angle), 0]
#else:
#    source_pos = [random.uniform(-4, 44), random.uniform(-4, 44), random.uniform(-50, 50)]

# Source position
#source_pos = [random.uniform(-4, 44), random.uniform(-4, 44), 0]
num_sources = 10
dimension = '2D'
shape_type = 'random'
x_min, x_max = 0, 40
y_min, y_max = 0, 40
radius = 5
n_photons = 3
source_pos = generate_source_points(num_sources, dimension,shape_type, x_min, x_max, y_min, y_max, radius)
#print(source_pos)
scatter_x = np.linspace(0, 40, 100)
scatter_y = np.linspace(0, 40, 100)
scatter_x, scatter_y = np.meshgrid(scatter_x, scatter_y)
scatter_z = -0 * scatter_x - 200  # Plane in xy at z = -200

absorber_x = np.linspace(-4, 44, 100)
absorber_y = np.linspace(-4, 44, 100)
absorber_x, absorber_y = np.meshgrid(absorber_x, absorber_y)
absorber_z = -0 * absorber_x - 300  # Plane in xy at z = -300

# Create figure
fig = go.Figure()

# Create grid lines for scatter plane
scatter_grid_lines = []
grid_size_scatter = 2
for i in range(0, 41, grid_size_scatter):
    scatter_grid_lines.append(go.Scatter3d(
        x=[i, i], y=[0, 40], z=[-200, -200],
        mode='lines', line=dict(color='rgb(255,182,193)', width=1), showlegend=False
    ))
    scatter_grid_lines.append(go.Scatter3d(
        x=[0, 40], y=[i, i], z=[-200, -200],
        mode='lines', line=dict(color='rgb(255,182,193)', width=1), showlegend=False
    ))

# Create grid lines for absorption plane
absorption_grid_lines = []
grid_size_absorption = 6
for i in range(-4, 45, grid_size_absorption):
    absorption_grid_lines.append(go.Scatter3d(
        x=[i, i], y=[-4, 44], z=[-300, -300],
        mode='lines', line=dict(color='gray', width=1), showlegend=False
    ))
    absorption_grid_lines.append(go.Scatter3d(
        x=[-4, 44], y=[i, i], z=[-300, -300],
        mode='lines', line=dict(color='gray', width=1), showlegend=False
    ))

# Add the grid lines to the figure
fig.add_traces(scatter_grid_lines)
fig.add_traces(absorption_grid_lines)

# Plot scatterer plane
fig.add_trace(go.Surface(x=scatter_x, y=scatter_y, z=scatter_z, opacity=0.7, showscale=False))

# Plot absorber plane
fig.add_trace(go.Surface(x=absorber_x, y=absorber_y, z=absorber_z, opacity=0.7, showscale=False))

# Plot source
for pos in source_pos:
    fig.add_trace(go.Scatter3d(x=[pos[0]], y=[pos[1]], z=[pos[2]], mode='markers', marker=dict(color='green'), name='Source'))

# Create arrays for the positions of hits on the 1st set of detectors
scat_pos = np.zeros((n_photons, 3))
absorb_pos = np.zeros((n_photons, 3))
data = []
thetas = []
azimuths = []
for pos in source_pos:
    for i in range(n_photons):
        scat_pos = [round(random.uniform(0, 40), 2), round(random.uniform(0, 40), 2), -200]
        scat_pos_mapped = [map_to_detector(scat_pos[0], 2), map_to_detector(scat_pos[1], 2), scat_pos[2]]
        angles = kn.calc_scattering_angles()
        theta = angles[0]
        azimuth = angles[1]
        absorb_pos = [scat_pos_mapped[0], scat_pos_mapped[1] + 10*np.cos(theta), scat_pos_mapped[2] + 10*np.cos(azimuth) - 100]
        absorb_pos_mapped = [map_to_detector(absorb_pos[0], 6), map_to_detector(absorb_pos[1], 6), absorb_pos[2]]
        #angle_with_x_axis = kn.angle_with_x_axis(theta, azimuth)
        #print("1",np.rad2deg(angle_with_x_axis))
       # angle_x = kn.angle_x(theta)
        #print("2",angle_x)
        e_energy = electron_energy(source_energy, theta)
        data_point = np.concatenate((pos, absorb_pos_mapped, scat_pos_mapped, [theta], [e_energy]))
        data.append(data_point)
        thetas.append(theta)
        azimuths.append(azimuth)
        # Plot scatterer to source line
        fig.add_trace(go.Scatter3d(x=[pos[0], scat_pos_mapped[0]], y=[pos[1], scat_pos_mapped[1]], z=[pos[2], scat_pos_mapped[2]], mode='lines', line=dict(color='blue')))
        #plot scatter to absorber line
        fig.add_trace(go.Scatter3d(x=[scat_pos_mapped[0], absorb_pos_mapped[0]], y=[scat_pos_mapped[1], absorb_pos_mapped[1]], z=[scat_pos_mapped[2], absorb_pos_mapped[2]], mode='lines', line=dict(color='red')))        # Plot scatterer to absorber line
plot_histogram(np.rad2deg(thetas))
#plot_histogram(np.rad2deg(azimuths))

# Set layout
fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'), title='Interactive Simulation')

# Show plot
fig.show()



In [20]:
print(data[:5])

[array([ 9.76000000e+00,  2.56000000e+00,  0.00000000e+00,  1.10000000e+01,
        2.55927184e+01, -2.90873245e+02,  1.12400000e+01,  1.79300000e+01,
       -2.00000000e+02,  5.36951880e-01,  9.88497187e+05]), array([ 9.76000000e+00,  2.56000000e+00,  0.00000000e+00,  1.00000000e+00,
        3.09092987e+01, -3.09241179e+02,  1.25000000e+00,  2.11800000e+01,
       -2.00000000e+02,  1.34787804e-01,  8.44117285e+05]), array([ 9.76000000e+00,  2.56000000e+00,  0.00000000e+00,  3.70000000e+01,
        4.88059711e+01, -3.07400378e+02,  3.60900000e+01,  3.94900000e+01,
       -2.00000000e+02,  1.97311750e-01,  9.20660375e+05]), array([ 2.61900000e+01,  3.61300000e+01,  0.00000000e+00,  7.00000000e+00,
        1.48957404e+01, -3.09925568e+02,  6.94000000e+00,  5.59000000e+00,
       -2.00000000e+02,  1.44527715e-01,  8.61609961e+05]), array([ 2.61900000e+01,  3.61300000e+01,  0.00000000e+00,  7.00000000e+00,
        2.08688692e+01, -2.93303851e+02,  6.48000000e+00,  1.07900000e+01,
       -2

In [10]:
for _ in range(5):
            angle = random.uniform(0, 2 * np.pi)
            elevation = random.uniform(-np.pi / 2, np.pi / 2)
            x = distance * np.cos(elevation) * np.cos(angle)
            y = distance * np.cos(elevation) * np.sin(angle)
            z = distance * np.sin(elevation)
            #source_positions.append([x, y, z])

NameError: name 'distance' is not defined

In [8]:
import pandas as pd
# Convert data to a DataFrame
columns = ['source_x', 'source_y', 'source_z', 'absorb_x', 'absorb_y', 'absorb_z', 
           'scat_x', 'scat_y', 'scat_z', 'theta', 'e_energy']
df = pd.DataFrame(data, columns=columns)

# Save DataFrame to CSV
df.to_csv('scattering_data.csv', index=False)

In [6]:
import matplotlib.pyplot as plt
def plot_histogram(scatter_angles):
    plt.hist(scatter_angles, bins=30, edgecolor='black')
    plt.xlabel('Scatter Angle (degrees)')
    plt.ylabel('Frequency')
    plt.title('Histogram of Scatter Angles')
    plt.show()


In [ ]:
scatter_angles = kn.calc_scattering_angles()
scatter_angles = np.rad2deg(scatter_angles)
plot_histogram(scatter_angles)
